In [1]:
# 從linebot 套件包裡引用 LineBotApi 與 WebhookHandler 類別
from linebot import (
    LineBotApi, WebhookHandler
)

# 生成實體物件
# line_bot_api = LineBotApi()
# handler = WebhookHandler()

In [2]:
# 載入json處理套件
import json

# 載入基礎設定檔
secretFile=json.load(open("./secret_key",'r'))
server_url=secretFile.get("server_url")


# channel_access_token是用於傳送封包去給line的認證使用類似這個是私鑰，而公鑰已經丟到line那邊，拿著這個就可以有VIP特權
# secret_key是用於當line傳送封包過來時確定line是否為本尊，沒有被仿冒
line_bot_api = LineBotApi(secretFile.get("channel_access_token"))
handler = WebhookHandler(secretFile.get("secret_key"))
menu_id = secretFile.get("rich_menu_id")


In [3]:
'''

製作文字與圖片的教學訊息

'''
# 將消息模型，文字收取消息與文字寄發消息 引入
from linebot.models import (
    MessageEvent, TextMessage, TextSendMessage, ImageSendMessage
)

# 消息清單
reply_message_list = [
TextSendMessage(text="CC102-Line考古題機器人。")
]
    

In [4]:
"""

  啟用伺服器基本樣板啟用伺服器基本 

"""

# 引用Web Server套件
from flask import Flask, request, abort , jsonify



# 引用無效簽章錯誤
from linebot.exceptions import (
    InvalidSignatureError
)


# 設定Server啟用細節
app = Flask(__name__,static_url_path = "/images" , static_folder = "./images/")


# 啟動server對外接口，使Line能丟消息進來
@app.route("/", methods=['POST'])
def callback():
    # get X-Line-Signature header value
    signature = request.headers['X-Line-Signature']

    # get request body as text
    body = request.get_data(as_text=True)
    app.logger.info("Request body: " + body)

    # handle webhook body
    try:
        handler.handle(body, signature)
    except InvalidSignatureError:
        abort(400)

    return 'OK'

In [5]:
'''

撰寫用戶關注時，我們要處理的商業邏輯

1. 取得用戶個資，並存回伺服器
2. 把先前製作好的自定義菜單，與用戶做綁定
3. 回應用戶，歡迎用的文字消息與圖片消息

'''


# 載入Follow事件
from linebot.models.events import (
    FollowEvent
)

# 載入requests套件
import requests


# 告知handler，如果收到FollowEvent，則做下面的方法處理
@handler.add(FollowEvent)
def reply_text_and_get_user_profile(event):
    
    # 取出消息內User的資料
    ################### 須作例外處理
    user_profile = line_bot_api.get_profile(event.source.user_id)
    print(user_profile.user_id)
        
    # 將用戶資訊做成合適Json

    user_info = {  
        "user_open_id":user_profile.user_id,
        "user_nick_name":user_profile.display_name,
        "user_status" : "",
        "user_img" : user_profile.picture_url,
        "user_register_menu" : menu_id
    }
    #將json傳回API Server
    Endpoint='http://192.168.122.100/users'   
    Header={'Content-Type':'application/json'}
    Response=requests.post(Endpoint,headers=Header,data=json.dumps(user_info))
    
    # 先暫時寫的，用來檢查錯誤情形，方便除錯
    print(Response)
    print(Response.text)
    
    # 將菜單綁定在用戶身上
    
    linkMenuEndpoint='https://api.line.me/v2/bot/user/%s/richmenu/%s' % (user_profile.user_id, menu_id)
    linkMenuRequestHeader={'Content-Type':'image/jpeg','Authorization':'Bearer %s' % secretFile["channel_access_token"]}
    lineLinkMenuResponse=requests.post(linkMenuEndpoint,headers=linkMenuRequestHeader)
   
    
    # 回覆文字消息與圖片消息
    line_bot_api.reply_message(
         event.reply_token,
         reply_message_list
    )

In [7]:
'''

執行此句，啟動Server，觀察後，按左上方塊，停用Server

'''

if __name__ == "__main__":
    app.run(host='0.0.0.0',port=5000)

 * Running on http://0.0.0.0:5000/ (Press CTRL+C to quit)
172.23.0.4 - - [16/Sep/2018 06:11:47] "POST / HTTP/1.1" 200 -


Ua5ec88edae803461e0c943ef27a0312e
<Response [200]>
{
  "status_describe": "success add user"
}



172.23.0.4 - - [16/Sep/2018 06:12:00] "POST / HTTP/1.1" 200 -
